# Homework exercise 2
## Deadline: upload to Moodle by 31 May 18:00 h

__Please submit your homework either as a Jupyter Notebook or using .py files.__

If you use .py files, please also include a PDF containing the output of your code and your explanations. Either way, the code needs to be in a form that can be easily run on another computer.

__Name 1:__ Bernd Prostmaier, 12004113

__Name 2:__ Mathias Broman, 11935158

__Name 3:__


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 1 June to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Selenium


__Yahoo Finance News Crawling__

Your task is to collect and organize articles from Yahoo Finance News, available at https://finance.yahoo.com/news

This will require you to use Selenium for at least two reasons:

* The site initially loads only partially such that scrolling is necessary to access additional news articles
* The news articles themselves, if they are long, are initially shown (and downloadable) in an abbreviated form. Browser navigation is necessary to click on a button such that the whole article is displayed.

There are about 200 news articles available at a time, stemming from various sources (e.g. Bloomberg, Reuters), and covering different topic areas (e.g. Business, World).

Note that you will need to use some features of Selenium - in particular scrolling - that are not discussed in the class notebook but documented elsewhere. 

You are asked to

* Download all of the about 200 news articles available from Yahoo Finance News
* Exclude the advertisements listed in between the news articles
* Try and remove everything that is not part of the article itself (e.g. the sidebar containing a list of popular articles and other content) and save the text of each article in a text file
* Create a DataFrame containing the following information (to the extent it is available) for each article
    * Title
    * Source (e.g. Bloomberg)
    * Time and date of publication
    * Name and path of the file containing the article's text that you saved
    
Please save this DataFrame so that you can later reuse the data set you created.

In [3]:
import requests
import selenium.common.exceptions
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import json
from selenium import webdriver
import re
import random
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


# Creation of folder for downloaded articles
dir_yahoo = 'Yahoo News Articles'
try:
    os.mkdir(dir_yahoo)
except:
    print('Folder already exists')

    
# Empty lists for appending information to the dataframe at the end
titles = []
sources = []
time_and_dates = []
urls = []
paths = []
texts = []


# Driver setup
options = Options()
options.headless = True
main_url = "https://finance.yahoo.com/news"
driver = webdriver.Chrome()
driver.implicitly_wait(10)

Folder already exists


In [4]:
# Initializes the webdriver
driver.get(main_url)


# Get rid of the popup window and maximize window
driver.find_element_by_name("agree").click()
driver.maximize_window()


# Scrolling down x pixels, y pixels at a time to load the full HTML code
x = 35000
y = 300
for i in range(0,x,y):
    driver.execute_script(f"window.scrollTo({str(i)}, {str(i+y)})")
    time.sleep(0.1)

    
# Compute the required number of iterations
total = len(driver.find_elements_by_css_selector("h3 > a"))
print(f'Found {total} links (Ads, charts and quotes included)')


# Scroll back to the top of the page
driver.execute_script("window.scrollTo(0, 0)")


# Iterates through all found links on the webpage
for i in range(0, total):
    
    try: 
        # Find the url at index i 
        news = driver.find_elements_by_css_selector("h3 > a")[i] 

        # Ad filter
        if "beap.gemini" in news.get_attribute("href") or news.get_attribute("class") == "Fz(s) Ell Fw(600) C($linkColor)" or '.com/video' in news.get_attribute('href'):
            continue # skip iteration, start with new i

        # Chart and quote filter
        if 'yahoo.com/quote' in news.get_attribute('href') or 'yahoo.com/chart' in news.get_attribute('href'):
            continue
        
        # Extract the url
        url = news.get_attribute("href")
        urls.append(url)     # Append the URL to a list
        
        # Extract and format the title
        title = news.text 
        
        # Remove unwanted title characters
        if "/" in title:
            title = title.replace("/", "")
        if "/'s" in title:
            title = title.replace("/'s", "")
            
        titles.append(title)   # Append the title to a list 
        
        # Click on the article
        news.click()

        # Getting information about time and source
        clock = driver.find_element_by_css_selector("div > time").text
        time_and_dates.append(clock)  # Appends the time and date to a list
        
        source = driver.find_element_by_css_selector("header > div > a > img").get_attribute("alt")       
        sources.append(source)        # Appends the source to a list
        
        ### Getting the text out of the article  
        # Some articles require a button to be clicked to view the full content
        if "Story continues" in driver.find_element_by_class_name("caas-body").text:
            driver.find_element_by_css_selector("button[class^='link rapid']").click()
            text = driver.find_element_by_class_name("caas-body").text
            time.sleep(0.4)
        else:
            text = driver.find_element_by_class_name("caas-body").text
 
        texts.append(text) # Appends the content of the article to a list
        time.sleep(0.3)

        # Download and save the article
        try:
            path = f'{os.getcwd()}/{dir_yahoo}/{str(title)}.txt'
            paths.append(path)
            
            my_file = open(path, "w+", encoding = "utf-8")
            my_file.write(str(title)+ "\n\n" + str(text))
            my_file.close()
        except Exception as msg: # Just in case if there is any error, inform us!
            print(msg)
            driver.back()
            continue

        # Go back to the news page and return the loop again with the next article
        driver.back() 
    # Stop at last index
    except IndexError:
        break

    # Sometimes after a few hundered clicks there appears a random add in front of the webpage
    # Since we don't know when or if this ad will show up, let us know and skip it
    except Exception as msg:
        del titles[-1] #| del because otherwise array dimensions in pandas df dont fit
        del urls[-1]  #| del because otherwise array dimensions in pandas df dont fit
        print(msg)
        continue
    
articles = pd.DataFrame({'Title': titles, 'Date Posted': time_and_dates, 'Source': sources, 'Content': texts, 'Path': paths, 'URL': urls})
# Saving our DataFrame for later use
articles.to_csv('Dataframe.csv',index=False)

driver.close()
                        
articles

Found 200 links (Ads, charts and quotes included)
Message: no such element: Unable to locate element: {"method":"css selector","selector":"div > time"}
  (Session info: chrome=90.0.4430.212)



,Title,Date Posted,Source,Content,Path,URL
0,China Mourns Death of Man Who Saved Millions F...,"Mon, May 24, 2021, 2:03 PM",Bloomberg,"(Bloomberg) -- The death of Yuan Longping, the...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/china-mourns-de...
1,U.S. Futures Gain; China Rhetoric Weighs on Me...,"Mon, May 24, 2021, 1:56 PM",Bloomberg,(Bloomberg) -- U.S. equity-index futures gaine...,/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/asia-stocks-sta...
2,UPDATE 1-Iran tells IAEA it is extending monit...,"Mon, May 24, 2021, 1:53 PM",Reuters,"(Adds quote, IAEA chief Grossi, detail)\nBy Pa...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/1-iran-tells-ia...
3,Thailand's Siam Commercial Bank to launch onli...,"Mon, May 24, 2021, 1:42 PM",Reuters,"BANGKOK (Reuters) - Thailand's oldest lender, ...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/thailands-siam-...
4,Virgin Galactic Shares Jump Following Successf...,"Mon, May 24, 2021, 1:38 PM",Bloomberg,(Bloomberg) -- Virgin Galactic Holdings Inc. c...,/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/branson-virgin-...
...,...,...,...,...,...,...
166,State Street to Move Some Staff to Ireland: Su...,"Sun, May 23, 2021, 12:50 PM",Bloomberg,(Bloomberg) --\nState Street Corp. will move a...,/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/state-street-mo...
167,Gaza truce holds as Israel admits Jewish visit...,"Sun, May 23, 2021, 12:15 PM",Reuters,"JERUSALEM/GAZA, May 23 (Reuters) - The Israel-...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/gaza-truce-hold...
168,UPDATE 1-India's capital Delhi to ease COVID-1...,"Sun, May 23, 2021, 12:07 PM",Reuters,"(Adds comments on Delhi reopening, India test ...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/1-indias-capita...
169,"Thailand to restrict cattle, buffalo movements...","Sun, May 23, 2021, 12:02 PM",Reuters,"BANGKOK, May 23 (Reuters) - Thailand has order...",/Users/mathiasbroman/Desktop/Summer 2021/Pytho...,https://finance.yahoo.com/news/thailand-restri...


From our point of view, there would have been 2 possibilities to solve the task. The first possibility would have been the *driver.get()* method and the other the *.click()* and *.back()* method.

Although the *driver.get()* method would probably have been easier, we decided to use the *.click()* method because it is much more runtime efficient. The code runs through super fast and gives us all the information we need. 

At the end we saved both our DataFrame (as .csv) and the text data (as .txt) at our local hard drive.

**So the test run worked as expected**